**FUNCIÓN ULTIMATE**

In [6]:
#Importación de librerias
import pandas as pd 
import numpy as np
import warnings
import re

**DECLARACIÓN DE FUNCIÓN**

In [14]:
def limpiar_correos(input_csv, output_csv, correos_eliminados_csv, wrong_words_regex):
    """
    Elimina correos electrónicos que contienen patrones no deseados (basados en regex) de un archivo CSV
    y elimina duplicados en el campo 'EMAIL' cuya frecuencia sea mayor a 4.

    Parámetros:
    input_csv (str): Ruta del archivo CSV de entrada.
    output_csv (str): Ruta del archivo CSV de salida.
    correos_eliminados_csv (str): Ruta del archivo CSV donde se guardarán los correos eliminados.
    wrong_words_regex (list): Lista de expresiones regulares con palabras o patrones a eliminar.

    Retorna:
    str: Mensaje de éxito.
    """
    # Ignorar advertencias de grupos de captura en regex
    warnings.filterwarnings("ignore", category=UserWarning)
    
    # Cargar el archivo
    df = pd.read_csv(input_csv, on_bad_lines='skip', low_memory=False, sep=",", encoding = 'latin1')

    # Eliminar valores nulos en 'EMAIL'
    print("SECCIÓN DE NULOS")
    print("Número de registros antes de eliminar valores nulos en 'EMAIL':", len(df))
    df.dropna(subset=['EMAIL'], inplace=True)
    print("Número de registros después de eliminar valores nulos en 'EMAIL':", len(df))
    print()

    # Limpiar la columna 'EMAIL'
    df['EMAIL'] = df['EMAIL'].str.lower().str.strip().str.replace(' ', '')

    # Filtrar correos no deseados usando regex
    combined_regex = '|'.join(wrong_words_regex)
    mask = df['EMAIL'].str.contains(combined_regex, regex=True)
    correos_eliminados = df[mask]['EMAIL'].tolist()
    df_clean = df[~mask]

    # Eliminar duplicados con frecuencia mayor a 4
    frecuencia_correos = df_clean.groupby('EMAIL').size()
    correos_repetidos = frecuencia_correos[frecuencia_correos > 4].index
    df_clean = df_clean[~df_clean['EMAIL'].isin(correos_repetidos)]

    print("SECCIÓN DE REGISTROS CON FRECUENCIAS ALTAS")
    print(f"Número de correos con frecuencia mayor a 4: {len(correos_repetidos)}")
    print(f"Número de filas eliminadas debido a duplicados con frecuencia mayor a 4: {len(df) - len(df_clean)}")
    print()

    # Limpiar la columna 'NOMBRES'
    if 'NOMBRES' in df_clean.columns:
        df_clean['NOMBRES'] = (
            df_clean['NOMBRES']
            .str.strip()
            .replace(['', ' ', '  ', '   '], np.nan, regex=False)
            .replace(r'[^a-zA-Z\s]', np.nan, regex=True)
            .replace('.', '')
            .fillna('ESTIMADO')
            .replace('MA', 'MARIA')
            .str.split(" ").str.get(0)
            
        )
        print("SECCIÓN DE LLENADO DE NOMBRES VACÍOS")
        print("Cantidad de nombres llenados con 'ESTIMADO':", (df_clean['NOMBRES'] == 'ESTIMADO').sum())
        print()
    else:
        print("Error: La columna 'NOMBRES' no existe en el DataFrame.")

    # Eliminar registros REPEP
    if 'REPEP' in df_clean.columns:
        print('SECCIÓN DE REPEP')
        print(f'El número de registros en el DataFrame antes de eliminar REPEP: {len(df_clean)}')
        repep = len(df_clean[df_clean['REPEP'] == 'Y'])
        print(f'El número de registros REPEP es de: {repep}')
        df_clean = df_clean[df_clean['REPEP'] != 'Y']
        print(f'El número de registros en el DataFrame después de eliminar REPEP: {len(df_clean)}')
        print()
    else:
        print("Error: La columna 'FLG_REPEP' no existe en el DataFrame.")

    # Guardar resultados
    print("SECCIÓN DE CORREOS ELIMINADOS POR REGEX")
    print(f"Número de correos eliminados por patrones no deseados: {len(correos_eliminados)}")
    print()

    pd.DataFrame(correos_eliminados, columns=['Correo Eliminado']).to_csv(correos_eliminados_csv, index=False)
    df_clean.to_csv(output_csv, index=False)

    return 'Felicidades, has limpiado tu base con éxito'

**LISTA REGEX**

In [8]:
wrong_words_regex = [
    # Errores de formato básico
    r'@@',                               # Doble arroba
    r'\.\.',                             # Doble punto
    r'\.com@',                           # ".com" seguido de arroba
    r'[\+\&:,"\' ]',                     # Caracteres especiales o espacios
    r'ñ',                                # Caracter "ñ"
    r'^[._\W]',                          # Correos que comienzan con punto, guion bajo o caracteres no alfanuméricos
    r'\.$',                              # Termina con un punto final
    r'@.*@',                             # Múltiples @ no consecutivos
    r'^[^@]*$',                          # No contiene @

    # Errores tipográficos comunes
    r'gm?ial|gnial|dumy|notene',         # Errores tipográficos comunes

    # Dominios específicos a excluir
    r'@(yopmail)',                       # Excluir solo el dominio "yopmail"

    # Palabras genéricas o placeholders
    r'(dummy|prueba|ejemplo|temporal)',  # Palabras genéricas o placeholders
    r'\bcorreoprueba\b',                 # Placeholder "correoprueba"

    # Frases que indican falta de correo (con variantes)
    r'\b(?:no[_.-]?tiene|sin[_.-]?correo|no[_.-]?tengo|no[_.-]?existe|no[_.-]?cuenta(?:con)?correo)\b',  # Captura "notiene", "no.tiene", "no_tiene", etc.
    r'\b(?:no[_.-]?aplica|no[_.-]?maneja[_.-]?correo|no[_.-]?brinda|no[_.-]?asigna[_.-]?correo)\b',      # Captura "no.aplica", "no_maneja_correo", etc.
    r'\b(?:no[_.-]?se[_.-]?lo[_.-]?sabe|no[_.-]?da|no[_.-]?desea|no[_.-]?dejo)\b',                      # Captura "no.se.lo.sabe", "no_da", etc.

    # Correos que comienzan con secuencias numéricas o letras específicas
    r'^(1\.21|1212|1122|121|12.1|1\.2)', # Correos que juegan con la secuencia de números 1 y 2
    r'^(618)',                           # Correos que juegan con la secuencia de números 6
    r'^(123|1234|12345)@',               # Empieza con 123@ (ampliado)
    r'^[zZ]+@',                          # Correos que solo contienen z antes del @ (ampliado)

    # Correos que comienzan con palabras o frases específicas
    r'^(sinusuario|demientras|notengo|virtual|facturas[-_]?izzi|cliente(?:nuevo|izzi)|izzi|aaa|om|lacasadelbrujo|nohaycorreo|ninguno)@.*$',  # Placeholders combinados
    r'^(manololo|sinmail|n|ngonzalez|generico|atencionalcliente|izzigenerico|p-ngonzalezg|a.empresarial)@.*$',  # Placeholders combinados
    r'^(ventas\d+clientes|xxx|generico\d+)@.*$',  # Placeholders
    r'^(jesus|jose)@.*$|^sincorreo\d+@.*$',  # Nombres de técnicos
    r'^(correo|123456|sin.correo|no.se.lo.sabe|wizzgenerico)@.*$',  # Placeholders
    r'^(abc|abc123|atc|sincontacto|iz.z.ot.el.e)@.*$',  # Placeholders

    # Correos que contienen "notiene" en cualquier parte (con variantes)
    r'[\w.-]*no[_.-]?tiene[\w.-]*@[\w.-]*',  # Captura "notiene", "no.tiene", "no_tiene", etc., en cualquier parte del correo
    r'@no[_.-]?tiene[\w.-]*',                # Captura "notiene", "no.tiene", "no_tiene", etc., después del @

    # Correos que comienzan con "nocuenta" o variantes
    r'^(nocuenta[\w-]+|nodejo[\w-]+|noda[\w-]+|nodesea[\w-]+)@',  # No cuenta

    # Correos con solo ceros o "1" antes del dominio
    r'^(0+|1)@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',

    # Correos con 1 o 2 dígitos antes del @
    r"^[a-zA-Z0-9]{1,2}@[\w.-]+\.[a-zA-Z]{2,}$",
]

**USO DE FUNCIÓN**

In [15]:
limpiar_correos(r'C:\Users\lgonzalezc\Documents\Trabajo\DB\MAX 12MESES\Base\REQMKT0402251200_CM_MAX.csv',
                r'C:\Users\lgonzalezc\Documents\Trabajo\DB\MAX 12MESES\Base\BD_CM_MAX_LIMPIA.csv',
                r'C:\Users\lgonzalezc\Documents\Trabajo\DB\MAX 12MESES\Base\BD_CM_MAX_CORREOS_ELIMINADOS.csv',
                wrong_words_regex)

SECCIÓN DE NULOS
Número de registros antes de eliminar valores nulos en 'EMAIL': 100276
Número de registros después de eliminar valores nulos en 'EMAIL': 100197

SECCIÓN DE REGISTROS CON FRECUENCIAS ALTAS
Número de correos con frecuencia mayor a 4: 4
Número de filas eliminadas debido a duplicados con frecuencia mayor a 4: 912

SECCIÓN DE LLENADO DE NOMBRES VACÍOS
Cantidad de nombres llenados con 'ESTIMADO': 195

SECCIÓN DE REPEP
El número de registros en el DataFrame antes de eliminar REPEP: 99285
El número de registros REPEP es de: 1682
El número de registros en el DataFrame después de eliminar REPEP: 97603

SECCIÓN DE CORREOS ELIMINADOS POR REGEX
Número de correos eliminados por patrones no deseados: 886



'Felicidades, has limpiado tu base con éxito'